<div style = "text_algin: center; font-size: 40px; font-weight: bold">
Phase 01: Collecting data
</div>

## Import necessary Python modules

In [1]:
import requests
import numpy as np
import pandas as pd 
from bs4 import BeautifulSoup
from selenium import webdriver

from selenium.webdriver.chrome.options import Options
import re

We will collect to type of data: The real estate for sale and the real eastate for rent

## Part 01: Collect real estate for sale data:

### 1. Initials:

Create an empty dataframe to store the data of the real estate. The dataframe will contain some features like: Address, Area, Price, Bedroom, Toilet, Floor, Furniture, Direction, Legal, Posting date, Expiry date, Ad type, Ad code

In [2]:
fields = [ 'Address', 'Area', 'Price', 'Bedroom', 'Toilet', 'Floor', 'Furniture', 'Direction','Legal', 'Posting date', 'Expiry date', 'Ad type', 'Ad code']

df = pd.DataFrame(columns = fields)

### 2. Collect the links to each real estate page:

The links are collected from multiple pages, with each page containing numerous links to apartments. These links have been stored in a file named "real_estate_for_sale_links.txt".     
The link to the homepage and the file name are provided below:

In [3]:
file_name = "../Data/real_estate_for_sale_links.txt"
url = 'https://batdongsan.com.vn/nha-dat-ban-tp-hcm'

We will move from the first page to the last page to collect the links, so the first thing should do is get the final page of the website.

In [5]:
def GetTotalPage(url):

    driver = webdriver.Chrome()
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    unprocessed_num_page = soup.find(class_ = 're__pagination-group')
    driver.quit()

    regex = r'[^\s]+'
    matchs = re.findall(regex, unprocessed_num_page.text)
    new_array = []
    for match in matchs:
        new_array.append(int(match.replace('.', '')))

    total_page = max(new_array)

    return total_page

Process the result to get the last page.

In [7]:
total_page = GetTotalPage(url)
total_page

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.200)
Stacktrace:
	GetHandleVerifier [0x00007FF6344282B2+55298]
	(No symbol) [0x00007FF634395E02]
	(No symbol) [0x00007FF6342505AB]
	(No symbol) [0x00007FF634230038]
	(No symbol) [0x00007FF6342B6BC7]
	(No symbol) [0x00007FF6342CA15F]
	(No symbol) [0x00007FF6342B1E83]
	(No symbol) [0x00007FF63428670A]
	(No symbol) [0x00007FF634287964]
	GetHandleVerifier [0x00007FF6347A0AAB+3694587]
	GetHandleVerifier [0x00007FF6347F728E+4048862]
	GetHandleVerifier [0x00007FF6347EF173+4015811]
	GetHandleVerifier [0x00007FF6344C47D6+695590]
	(No symbol) [0x00007FF6343A0CE8]
	(No symbol) [0x00007FF63439CF34]
	(No symbol) [0x00007FF63439D062]
	(No symbol) [0x00007FF63438D3A3]
	BaseThreadInitThunk [0x00007FF83F86257D+29]
	RtlUserThreadStart [0x00007FF8414EAA58+40]


Function to collect the links to each real estate:

In [6]:
def CollectRealEstateForSaleLink (url, page):
    with open (file_name, 'w') as file:

        for i in range(1, page + 1):
            driver = webdriver.Chrome()
            driver.get(url+'/p'+ str(i))
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            apartment_list = soup.find_all("a", class_= "js__product-link-for-product-id")

            for apartment in apartment_list:
                link = 'https://batdongsan.com.vn/'+ apartment['href']
                file.write(link)
                file.write('\n')

            driver.quit()

In [ ]:
CollectRealEstateForSaleLink (url, total_page)

Read the "real_estate_for_sale_links.txt" file and store all the links to a lists

In [4]:
urls = []
with open(file_name, 'r') as file:
    for line in file.readlines():
            urls.append(line)

### 3. Access to each link and colllect the data:

- A dictionary to rename the features collected from the webiste:

In [3]:
key_map = {
    'Địa chỉ:' : 'Address',
    'Diện tích': 'Area',
    'Mức giá': 'Price',
    'Hướng nhà': 'Direction',
    'Số tầng': 'Floor',
    'Số phòng ngủ': 'Bedroom',
    'Số toilet': 'Toilet',
    'Pháp lý': 'Legal',
    'Nội thất': 'Furniture',
    'Ngày đăng': 'Posting date',
    'Ngày hết hạn': 'Expiry date',
    'Loại tin': 'Ad type',
    'Mã tin': 'Ad code'
}

- Function to extract the features from a link:

In [6]:
def ExtractDataFromLink(url):
    driver = webdriver.Chrome()
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        #Create a dicitonary to store the feature
        features = {}

        # real_estate_list = soup.find_all("a", class_= "js__product-link-for-product-id")
        # if real_estate_list:
        #     for item in real_estate_list:
        #         link = 'https://batdongsan.com.vn/'+ apartment['href']
        #         features = ExtractDataFromLink(link)
        # else:

        # get the address
        address = soup.find(class_ = "re__pr-short-description js__pr-address")

        if address == None:
            return features
        else:
            features['Address'] = address.text

        # Get all the features
        feature_items = soup.find_all('div', class_ = 're__pr-specs-content-item')
        for item in feature_items:
            title = item.find(class_ = 're__pr-specs-content-item-title').text.strip()
            value = item.find(class_ = 're__pr-specs-content-item-value').text.strip()
            features[title] = value

        date_items = soup.find_all( 'div', class_ = "re__pr-short-info-item js__pr-config-item")
        for item in date_items:
            title = item.find('span', class_ = 'title').text.strip()
            value = item.find('span',class_ = 'value').text.strip()
            features[title] = value
    
        keys_to_change = list(features.keys())
        for key in keys_to_change:
            if key in key_map:
                features[key_map[key]] = features[key]
    finally:
        driver.quit()

    return features

Iterate over the urls list and extract the data, which is stored in the previously defined DataFrame

In [7]:
for url in urls[ 1184 : 2000]:
        features = ExtractDataFromLink(url)
        if features:
                new_row = pd.Series(features, index = fields)
                new_row_df = pd.DataFrame([new_row])
                # df = pd.concat([df, new_row_df], ignore_index = True)
                new_row_df.to_csv('../Data/real_estate_for_sale.csv', mode='a', header=False, index=False, encoding='utf-8')
        else:
                print ("hihi")


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.200)
Stacktrace:
	GetHandleVerifier [0x00007FF7618882B2+55298]
	(No symbol) [0x00007FF7617F5E02]
	(No symbol) [0x00007FF7616B05AB]
	(No symbol) [0x00007FF761690038]
	(No symbol) [0x00007FF761716BC7]
	(No symbol) [0x00007FF76172A15F]
	(No symbol) [0x00007FF761711E83]
	(No symbol) [0x00007FF7616E670A]
	(No symbol) [0x00007FF7616E7964]
	GetHandleVerifier [0x00007FF761C00AAB+3694587]
	GetHandleVerifier [0x00007FF761C5728E+4048862]
	GetHandleVerifier [0x00007FF761C4F173+4015811]
	GetHandleVerifier [0x00007FF7619247D6+695590]
	(No symbol) [0x00007FF761800CE8]
	(No symbol) [0x00007FF7617FCF34]
	(No symbol) [0x00007FF7617FD062]
	(No symbol) [0x00007FF7617ED3A3]
	BaseThreadInitThunk [0x00007FF83F86257D+29]
	RtlUserThreadStart [0x00007FF8414EAA58+40]


### 4. Store the collected data to csv file:

In [21]:
df.to_csv('../Data/real_estate_for_sale.csv', index = False)

In [10]:
real_estate_df = pd.read_csv('../Data/real_estate_for_sale.csv')

In [12]:
real_estate_df.shape()

,Address,Area,Price,Bedroom,Toilet,Floor,Furniture,Direction,Legal,Posting date,Expiry date,Ad type,Ad code
0,"Dự án Lumiere Riverside, Đường Xa Lộ Hà Nội, P...",77 m²,"6,1 tỷ",2 phòng,2 phòng,NaN,NaN,NaN,NaN,27/11/2023,07/12/2023,Tin VIP Kim Cương,38645401
1,"Dự án Lumiere Riverside, Đường Xa Lộ Hà Nội, P...",76 m²,"6,2 tỷ",2 phòng,2 phòng,NaN,Đầy đủ,NaN,Hợp đồng mua bán,23/11/2023,30/11/2023,Tin thường,38393009
2,"Dự án Lumiere Riverside, Đường Xa Lộ Hà Nội, P...",72 m²,Thỏa thuận,2 phòng,2 phòng,NaN,NaN,NaN,NaN,28/11/2023,05/12/2023,Tin VIP Kim Cương,38657478
3,"Lumiere Riverside, 628, Đường Xa Lộ Hà Nội, Ph...","77,1 m²","6,6 tỷ",2 phòng,2 phòng,NaN,Cơ bản,NaN,Hợp đồng mua bán,29/11/2023,06/12/2023,Tin VIP Kim Cương,38403632
4,"Dự án The Privia, Đường An Dương Vương, Phường...","67,74 m²","3,5 tỷ",2 phòng,2 phòng,NaN,"Bàn giao cơ bản các thiết bị Châu Âu cao cấp, ...",NaN,HĐMB,29/11/2023,06/12/2023,Tin VIP Kim Cương,38654457
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,"Đường Nhuận Đức, Xã Nhuận Đức, Củ Chi, Hồ Chí ...",4.250 m²,"6,3 triệu/m²",NaN,NaN,NaN,NaN,NaN,Sổ hồng riêng,28/11/2023,13/12/2023,Tin VIP Vàng,38451882
94,"Dự án Akari City Nam Long, Đường Võ Văn Kiệt, ...",60 m²,Thỏa thuận,2 phòng,1 phòng,NaN,Cơ bản,NaN,Sổ đỏ/ Sổ hồng,28/11/2023,05/12/2023,Tin VIP Vàng,38653484
95,"Dự án HaDo Centrosa Garden, Đường 3/2, Phường ...",86 m²,"6,6 tỷ",2 phòng,2 phòng,NaN,NaN,NaN,NaN,23/11/2023,08/12/2023,Tin VIP Vàng,35489279
96,"Dự án KDC Phú Mỹ, Đường Hoàng Quốc Việt, Phườn...",126 m²,"20,7 tỷ",4 phòng,5 phòng,3 tầng,NaN,NaN,Có sổ,28/11/2023,08/12/2023,Tin VIP Vàng,38526845


In [4]:
import cloudscraper
from bs4 import BeautifulSoup as bs
url = 'https://batdongsan.com.vn/nha-dat-ban-tp-hcm'
url_list = []

scraper = cloudscraper.create_scraper(delay=10, browser="chrome") 
content = scraper.get(url)

soup = bs(content.text, 'html.parser')
num_of_page = soup.find_all('a', class_='-numberre__pagination')

# num_of_page = int(num_of_page)

page = int(num_of_page[-1]['pid'])
print (page)







2797


In [27]:
# Get url list in first page
a_tags_list = soup.find_all('a', class_='js__product-link-for-product-id')
for a_tag in a_tags_list:
    url_list.append('https://batdongsan.com.vn/' + a_tag.get('href'))

url_list

['https://batdongsan.com.vn//ban-can-ho-chung-cu-duong-vo-van-kiet-phuong-cau-kho-prj-zenity/duy-nhat-1-penthouse-duplex-trung-tam-quan-1-cat-lo-41-moi-100-ban-giao-full-noi-that-pr38720367',
 'https://batdongsan.com.vn//ban-can-ho-chung-cu-duong-vo-van-kiet-phuong-cau-kho-prj-zenity/cat-lo-40-3pn-116m2-trung-tam-quan-1-gia-goc-18-ty-gia-ban-chua-den-10-ty-view-bitexco-q1-pr38693652',
 'https://batdongsan.com.vn//ban-can-ho-chung-cu-duong-vo-van-kiet-phuong-cau-kho-prj-zenity/ban-gap-dinh-ada-full-noi-that-da-co-so-ng-70-trieu-m2-da-thanh-toan-het-pr38481731',
 'https://batdongsan.com.vn//ban-can-ho-chung-cu-duong-xa-lo-ha-noi-phuong-an-phu-prj-lumiere-riverside/thanh-ly-8-2-phong-ngu-do-mat-kha-nang-thanh-toan-lien-he-mr-hieu-pr38393009',
 'https://batdongsan.com.vn//ban-can-ho-chung-cu-duong-vo-van-kiet-phuong-cau-kho-prj-zenity/truc-tiep-cdt-capitaland-chiet-khau-40-2pn-tu-7-ty-3pn-tu-9-ty-nhan-nha-ngay-moi-100-pr38734339',
 'https://batdongsan.com.vn//ban-can-ho-chung-cu-duong-xa-l

In [30]:
page = '/p'
num_page = 2772
for num_page in range(2, int(num_page) + 1):
    content = scraper.get(url + page + str(num_page)).text
    soup = bs(content, 'html.parser')
    a_tags_list = soup.find_all('a', class_='js__product-link-for-product-id')
    for a_tag in a_tags_list:
        url_list.append('https://batdongsan.com.vn/' + a_tag.get('href'))

In [5]:
file_name = 'real_estate.txt'

with open(file_name, 'w') as file:
    for line in url_list:
        file.write(line + "\n")

In [5]:
def ExtractDataFromLink2(url, scraper, headers):

    content = scraper.get(url, timeout=30, headers=headers)

    content.encoding = 'utf-8'
    soup = bs(content.text, 'html.parser')

    #Create a dicitonary to store the feature
    features = {}
    final_features = {}

    # get the address
    address = soup.find(class_ = "re__pr-short-description js__pr-address")

    if address is None:
        print(content)
        address = soup.find(class_ = 're__address-title js__product-address')
        if address is None:
            return final_features
        features['Địa chỉ:'] = address.text.strip()
        print("hihi")

        feature_items = soup.find_all('div', class_ = 're__pr-specs-content-item')
        for item in feature_items:
            title = item.find(class_ = 're__pr-specs-content-item-title').text.strip()
            value = item.find(class_ = 're__pr-specs-content-item-value').text.strip()
            features[title] = value

        date_items = soup.find_all( 'ul', class_ = "re__product-info")
        for item in date_items:
            title = item.find('span', class_ = 're__sp1').text.strip()
            value = item.find('span',class_ = 're__sp3').text.strip()
            features[title] = value

        keys_to_change = list(features.keys())

        for key in keys_to_change:
            if key in key_map:
                final_features[key_map[key]] = features[key]
        return final_features
    
    else:
        features['Địa chỉ:'] = address.text.strip()

        # Get all the features
        feature_items = soup.find_all('div', class_ = 're__pr-specs-content-item')
        for item in feature_items:
            title = item.find(class_ = 're__pr-specs-content-item-title').text.strip()
            value = item.find(class_ = 're__pr-specs-content-item-value').text.strip()
            features[title] = value

        date_items = soup.find_all( 'div', class_ = "re__pr-short-info-item js__pr-config-item")
        for item in date_items:
            title = item.find('span', class_ = 'title').text.strip()
            value = item.find('span',class_ = 'value').text.strip()
            features[title] = value

        keys_to_change = list(features.keys())

        for key in keys_to_change:
            if key in key_map:
                final_features[key_map[key]] = features[key]
        return final_features
   

In [6]:
urls = []
with open('real_estate.txt', 'r') as file:
    for line in file.readlines():
            urls.append(line)



In [281]:

url = 'https://batdongsan.com.vn/ban-can-ho-chung-cu-duong-nguyen-huu-canh-phuong-22-prj-saigon-pearl/tin-that-hinh-that-100-sapphire-dt-141m2-gia-8-7-ty-tang-cao-noi-that-go-tu-nhien-100-pr37305304'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

scraper = cloudscraper.create_scraper(delay=10, browser="chrome") 
features = ExtractDataFromLink2(url, scraper, headers)

url1 = urls[15766]

print (url1[0:-1])



https://batdongsan.com.vn//ban-can-ho-chung-cu-duong-mai-chi-tho-phuong-thu-thiem-prj-empire-city-thu-thiem/-3pn-dac-biet-goc-ban-nha-dep-so-huu-view-truc-dien-song-vinh-cuu-3-phong-ngu-gia-30-ty-pr37304422


In [7]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
scraper = cloudscraper.create_scraper(delay=10, browser="chrome") 
for url1 in urls[51144: 55440]:
        features = ExtractDataFromLink2(url1[0:-1], scraper, headers)
        print(features)
        if features:
                new_row = pd.Series(features, index = fields)
                new_row_df = pd.DataFrame([new_row])
                df = pd.concat([df, new_row_df], ignore_index = True)
                new_row_df.to_csv('../Data/real_estate.csv', mode='a', header=False, index=False, encoding='utf-8')
        else:
                # break;
                
                print ("haha")
     

{'Address': 'Vinhomes Grand Park, 88, Đường Phước Thiện, Phường Long Bình, Quận 9, Hồ Chí Minh', 'Area': '68 m²', 'Price': '2,69 tỷ', 'Bedroom': '2 phòng', 'Toilet': '2 phòng', 'Furniture': 'Cơ bản', 'Posting date': '28/11/2023', 'Expiry date': '08/12/2023', 'Ad type': 'Tin thường', 'Ad code': '38655911'}
{'Address': 'Dự án The Grande, Phường Tân Phú, Quận 7, Hồ Chí Minh', 'Area': '89 m²', 'Price': '5,55 tỷ', 'Direction': 'Đông - Nam', 'Bedroom': '2 phòng', 'Toilet': '2 phòng', 'Legal': 'Sổ cầm ngân hàng', 'Furniture': 'Sang xịn', 'Posting date': '28/11/2023', 'Expiry date': '08/12/2023', 'Ad type': 'Tin thường', 'Ad code': '38584874'}
{'Address': 'Đường 10, Phường Hiệp Bình Chánh, Thủ Đức, Hồ Chí Minh', 'Area': '70 m²', 'Price': '6,5 tỷ', 'Floor': '3 tầng', 'Bedroom': '4 phòng', 'Toilet': '4 phòng', 'Legal': 'Sổ đỏ/ Sổ hồng', 'Furniture': 'Đầy đủ', 'Posting date': '28/11/2023', 'Expiry date': '28/12/2023', 'Ad type': 'Tin thường', 'Ad code': '37551673'}
{'Address': 'Đường Nguyễn Văn Đ

C:\Users\DELL\AppData\Local\Temp\ipykernel_25920\4180967528.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = bs(content.text, 'html.parser')


<Response [503]>
{}
haha
{'Address': 'Xã An Thới Đông, Cần Giờ, Hồ Chí Minh', 'Area': '3.900 m²', 'Price': '14 triệu/m²', 'Legal': 'Sổ đỏ/ Sổ hồng', 'Posting date': '28/11/2023', 'Expiry date': '08/12/2023', 'Ad type': 'Tin thường', 'Ad code': '38653932'}
{'Address': 'Dự án Riverside Residence, Đường Nguyễn Lương Bằng, Phường Tân Phú, Quận 7, Hồ Chí Minh', 'Area': '143 m²', 'Price': '10,5 tỷ', 'Direction': 'Đông', 'Bedroom': '3 phòng', 'Toilet': '2 phòng', 'Legal': 'Đã có sổ.', 'Furniture': 'Full nội thất cao cấp.', 'Posting date': '28/11/2023', 'Expiry date': '13/12/2023', 'Ad type': 'Tin thường', 'Ad code': '38653315'}
{'Address': 'Dự án Phố Đông - Hoa Sen, Đường D6, Phường Phước Long B, Quận 9, Hồ Chí Minh', 'Area': '60 m²', 'Price': '1,63 tỷ', 'Direction': 'Tây - Bắc', 'Bedroom': '2 phòng', 'Toilet': '1 phòng', 'Legal': 'Đã có sổ hồng', 'Furniture': 'Để lại một số nội thất', 'Posting date': '09/12/2023', 'Expiry date': '19/12/2023', 'Ad type': 'Tin thường', 'Ad code': '38536844'}
{

In [171]:
df.to_csv('../Data/real_estate.csv', index = False)

In [33]:
print(content.text)


<!DOCTYPE html>
<html>
<head>
    <link rel="preconnect" href="https://staticfile.batdongsan.com.vn" />
    <link rel="dns-prefetch" href="https://staticfile.batdongsan.com.vn" />
    <link rel="preconnect" href="https://file4.batdongsan.com.vn" />
    <link rel="dns-prefetch" href="https://file4.batdongsan.com.vn" />




        <script>
            window.unlFeatures = [{"f":"cplus-5252_ldp-agent-profile","v":"disabled","s":"0"},{"f":"cplus-7758_remaps-location","v":"cplus-7758_remaps-location_sg","s":"1"},{"f":"cplus-7758_remaps-location_sg","v":"cplus-7758_remaps-location_sg_disable","s":"1"},{"f":"cplus-6007_kyc-for-pricing-history","v":"cplus-6007_kyc-for-pricing-history_lock-1_kyc-1","s":"1"},{"f":"cplus-5593_change-link-to-pa","v":"cplus-5593_change-link-to-pa_v1","s":"1"},{"f":"cplus-6007_pricing-history-user","v":"cplus-6007_pricing-history-user_hold-out","s":"1"},{"f":"cplus-6007_pricing-history","v":"cplus-6007_pricing-history_disable-experiment","s":"1"},{"f":"cplus-7244_

## Part 02: Collect real estate for rent data:


# Finish collecting data